In [ ]:
import torch
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))
print(torch.cuda.get_device_properties(0))
print(torch.cuda.get_device_capability(0))


In [2]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torchaudio , librosa
import soundfile as sf
import time
import torch
from transformers import VitsTokenizer, VitsModel, set_seed
import scipy
from IPython.display import Audio

c:\Users\Angry Nerd\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# load model and processor
processor = WhisperProcessor.from_pretrained("openai/whisper-medium")
tokenizer = VitsTokenizer.from_pretrained("facebook/mms-tts-eng")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
tokenizer_config.json: 100%|██████████| 287/287 [00:00<?, ?B/s] 
c:\Users\Angry Nerd\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Angry Nerd\.cache\huggingface\hub\models--facebook--mms-tts-eng. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.m

In [4]:
#device="cuda:0" if torch.cuda_is_available() else "cpu"
#model_whisper = WhisperForConditionalGeneration.from_pretrained("openai/whisper-medium").cuda()
model_whisper = WhisperForConditionalGeneration.from_pretrained("openai/whisper-medium")

In [5]:
#model_vits = VitsModel.from_pretrained("facebook/mms-tts-eng").cuda()
model_vits = VitsModel.from_pretrained("facebook/mms-tts-eng")

config.json: 100%|██████████| 1.64k/1.64k [00:00<00:00, 69.4kB/s]
model.safetensors: 100%|██████████| 145M/145M [03:30<00:00, 689kB/s]  
Some weights of the model checkpoint at facebook/mms-tts-eng were not used when initializing VitsModel: ['flow.flows.0.wavenet.in_layers.0.weight_g', 'flow.flows.0.wavenet.in_layers.0.weight_v', 'flow.flows.0.wavenet.in_layers.1.weight_g', 'flow.flows.0.wavenet.in_layers.1.weight_v', 'flow.flows.0.wavenet.in_layers.2.weight_g', 'flow.flows.0.wavenet.in_layers.2.weight_v', 'flow.flows.0.wavenet.in_layers.3.weight_g', 'flow.flows.0.wavenet.in_layers.3.weight_v', 'flow.flows.0.wavenet.res_skip_layers.0.weight_g', 'flow.flows.0.wavenet.res_skip_layers.0.weight_v', 'flow.flows.0.wavenet.res_skip_layers.1.weight_g', 'flow.flows.0.wavenet.res_skip_layers.1.weight_v', 'flow.flows.0.wavenet.res_skip_layers.2.weight_g', 'flow.flows.0.wavenet.res_skip_layers.2.weight_v', 'flow.flows.0.wavenet.res_skip_layers.3.weight_g', 'flow.flows.0.wavenet.res_skip_layers.3.w

In [6]:
print(model_whisper.device)
print(model_vits.device)

cpu
cpu


In [7]:
def resample(old,resampled):
    y, sr = librosa.load(old)
    new_audio = librosa.resample(y, orig_sr=sr, target_sr=16000)
    sf.write(resampled, new_audio, 16000)
    

In [8]:
original = "_assets/arabic_test.wav"
resampled = "_assets/resampled_audio.wav"
resample(original,resampled)

In [9]:
forced_decoder_ids = processor.get_decoder_prompt_ids(language="english", task="translate")

audio, sample_rate = torchaudio.load(resampled)

In [10]:
# Convert the audio tensor to a NumPy array (Whisper expects NumPy arrays)
audio_np = audio.numpy()
input_features = processor(audio_np, sampling_rate=sample_rate, return_tensors="pt").input_features 
#input_features = input_features.to(torch.device('cuda'))

In [11]:
# generate token ids
start_time = time.time()  # Record the start time
predicted_ids = model_whisper.generate(input_features, forced_decoder_ids=forced_decoder_ids)

translation = processor.batch_decode(predicted_ids, skip_special_tokens=True)


end_time = time.time()  # Record the end time
print(translation)
# Calculate the elapsed time
elapsed_time = end_time - start_time
print(f"Process took {elapsed_time:.2f} seconds.")


[' 1, 2, 3, test the artificial intelligence model, especially in the project of mobility.']
Process took 68.63 seconds.


In [12]:
inputs = tokenizer(text=translation, return_tensors="pt")
#inputs= inputs.to(torch.device('cuda'))

set_seed(555)  # make deterministic

with torch.no_grad():
   outputs = model_vits(**inputs)

waveform = outputs.waveform[0].to("cpu")

In [13]:
print(translation)
Audio(waveform, rate=model_vits.config.sampling_rate)

[' 1, 2, 3, test the artificial intelligence model, especially in the project of mobility.']
